# KNN with sklearn - Lab

## Introduction

In this lab, we'll learn how to use sklearn's implementation of a KNN classifier  on some real world datasets!

## Objectives

You will be able to:

* Use KNN to make classification predictions on a real-world dataset
* Perform a parameter search for 'k' to optimize model performance
* Evaluate model performance and interpret results

## Getting Started

In this lab, we'll make use of sklearn's implementation of the **_K-Nearest Neighbors_** algorithm. We'll use it to make predictions on the Titanic dataset. 

We'll start by importing the dataset, and then deal with preprocessing steps such as removing unnecessary columns and normalizing our dataset.

You'll find the titanic dataset stored in the `titanic.csv` file. In the cell below:

* Import pandas and set the standard alias.
* Read in the data from `titanic.csv` and store it in a pandas DataFrame. 
* Print the head of the DataFrame to ensure everything loaded correctly.

In [1]:
import pandas as pd
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Great! Now, we'll preprocess our data to get it ready for use with a KNN classifier.

## Preprocessing Our Data

This stage should be pretty familiar to you by now. Although it's not the fun part of machine learning, it's good practice to get used to it.  Although it isn't as fun or exciting as training machine learning algorithms, it's a very large, very important part of the Data Science Process. As a Data Scientist, you'll often spend the majority of your time wrangling and preprocessing, just to get it ready for use with supervised learning algorithms. 

Since you've done this before, you should be able to do this quite well yourself without much hand holding by now. 

In the cells below, complete the following steps:

1. Remove unnecessary columns (PassengerId, Name, Ticket, and Cabin).
2. Convert `Sex` to a binary encoding, where female is `0` and male is `1`.
3. Detect and deal with any null values in the dataset. 
    * For `Age`, replace null values with the median age for the dataset. 
    * For `Embarked`, drop the rows that contain null values
4. One-Hot Encode categorical columns such as `Embarked`.
5. Store our target column, `Survived`, in a separate variable and remove it from the DataFrame. 

In [2]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True, axis=1)

In [3]:
criteria = (df['Sex'] == 'female')
df.loc[ criteria, 'Sex'] = 0
df.loc[~criteria, 'Sex'] = 1

In [4]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [5]:
df.Age.fillna(df['Age'].mean(), inplace=True)
df.dropna(axis =0,subset=['Embarked'], inplace=True)

In [6]:
df.isna().sum(),df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
Survived    889 non-null int64
Pclass      889 non-null int64
Sex         889 non-null int64
Age         889 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(5), object(1)
memory usage: 62.5+ KB


(Survived    0
 Pclass      0
 Sex         0
 Age         0
 SibSp       0
 Parch       0
 Fare        0
 Embarked    0
 dtype: int64, None)

In [7]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [8]:
df_hot = pd.get_dummies(df)
df_hot.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [9]:
target = df_hot.Survived
df_hot.drop('Survived', inplace=True, axis=1)

## Normalizing Our Data

Good job preprocessing our data! This can seem tedious, but its a very important foundational skill in any Data Science toolbox. The final step we we'll take in our preprocessing efforts is to **_Normalize_** our data. Recall that normalization (also sometimes called **_Standardization_** or **_Scaling_**) means making sure that all of our data is represented at the same scale.  The most common way to do this is to convert all numerical values to z-scores. 

Since KNN is a distance-based classifier, data on different scales and negatively affect the results of our model! Predictors on much larger scales will overwhelm data with much smaller scales, because euclidean distance is going to treat them as the same.

To scale our data, we'll make use of the `StandardScaler` object found inside the `sklearn.preprocessing` module. 

In the cell below:

* Import and instantiate a `StandardScaler` object. 
* Use the scaler's `.fit_transform()` method to create a scaled version of our dataset. 
* The result returned by the `fit_transform` call will be a numpy array, not a pandas DataFrame. Create a new pandas DataFrame out of this object called `scaled_df`. To set the column names back to their original state, set the `columns` parameter to `one_hot_df.columns`.
* Print out the head of `scaled_df` to ensure everything worked correctly.

In [10]:
# Dont forget to import!
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scaled = scale.fit_transform(df_hot)
scaled_df = pd.DataFrame(scaled, columns=df_hot.columns)

scaled_df.head()

/Users/michaelmoravetz/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/michaelmoravetz/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0.825209,0.735342,-0.590495,0.431350,-0.474326,-0.500240,-0.482711,-0.307941,0.616794
1,-1.572211,-1.359911,0.643971,0.431350,-0.474326,0.788947,2.071634,-0.307941,-1.621287
2,0.825209,-1.359911,-0.281878,-0.475199,-0.474326,-0.486650,-0.482711,-0.307941,0.616794
3,-1.572211,-1.359911,0.412509,0.431350,-0.474326,0.422861,-0.482711,-0.307941,0.616794
4,0.825209,0.735342,0.412509,-0.475199,-0.474326,-0.484133,-0.482711,-0.307941,0.616794


You may have noticed that the scaler also scaled our binary/one-hot encoded columns, too! Although it doesn't look as pretty, this has no negative effect on our model. Each 1 and 0 have been replaced with corresponding decimal values, but each binary column still only contains 2 values, meaning the overall information content of each column has not changed. 

#### Creating Training and Testing Sets

Now that we've preprocessed our data, the only step remaining is to split our data into training and testing sets. 

In the cell below:

* Import `train_test_split` from the `sklearn.model_selection` module
* Use `train_test_split` to split our data into training and testing sets, with a `test_size` of `0.25`.

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_df, target, test_size= .25)

## Creating and Fitting our KNN Model

Now that we've preprocessed our data successfully, it's time for the fun stuff--let's create a KNN classifier and use it to make predictions on our dataset!  Since you've got some experience on this part from when we built our own model, we won't hold your hand through section. 

In the cells below:

* Import `KNeighborsClassifier` from the `sklearn.neighbors` module.
* Instantiate a classifier. For now, we'll just use the default parameters. 
* Fit the classifier to our training data/labels
* Use the classifier to generate predictions on our testing data. Store these predictions inside the variable `test_preds`.

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
preds = knn.predict(X_test)

Now, in the cells below, import all the necessary evaluation metrics from `sklearn.metrics` abd then complete the following `print_metrics()` function so that it prints out **_Precision, Recall, Accuracy,_** and **_F1-Score_** when given a set of `labels` and `preds`. 

Then, use it to print out the evaluation metrics for our test predictions stored in `test_preds`, and the corresponding labels in `y_test`.

In [13]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [14]:
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds)))
    print("Recall Score: {}".format(recall_score(labels, preds)))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds)))
    
print_metrics(y_test, preds)

Precision Score: 0.7857142857142857
Recall Score: 0.7333333333333333
Accuracy Score: 0.8430493273542601
F1 Score: 0.7586206896551724


**_QUESTION:_** Interpret each of the metrics above, and explain what they tell us about our model's capabilities. If you had to pick one score to best describe the performance of the model, which would you choose? Explain your answer.

Write your answer below this line:
________________________________________________________________________________
    Precision tells us how often our model was correct at preedicting someone survivef. 71% of the time it was correct witht htat.  Recall tells us how many survivors our model classified correctly. accuracy tell us the overall percentage of correct predictions the model made. F1 is the mean precision and recall. F1 is the prefereable metric for juding a model, accuracy is the next preferable method. for multicategorical models accuracy IS the best. 


## Improving Model Performance

Our overall model results are better than random chance, but not by a large margin. For the remainder of this notebook, we'll focus on improving model performance. This is also a big part of the Data Science Process--your first fit is almost never your best. Modeling is an **_iterative process_**, meaning that we should make small incremental changes to our model and use our intuition to see if we can improve the overall performance. 

First, we'll start off by trying to find the optimal number of neighbors to use for our classifier. To do this, we'll write a quick function that iterates over multiple values of k and finds the one that returns the best overall performance. 

In the cell below, complete the `find_best_k()` function.  This function should:

* take in six parameters:
    * `X_train`, `y_train`, `X_test`, and  `y_test`
    * `min_k` and `max_k`. Set these to `1` and `25`, by default
* Create two variables, `best_k` and `best_score`
* Iterate through every **_odd number_** between `min_k` and `max_k + 1`. 
* For each iteration:
    * Create a new KNN classifier, and set the `n_neighbors` parameter to the current value for k, as determined by our loop.
    * Fit this classifier to the training data.
    * Generate predictions for `X_test` using the fitted classifier.
    * Calculate the **_F1-score_** for these predictions.
    * Compare this F1-score to `best_score`. If better, update `best_score` and `best_k`.
* Once it has checked every value for `k`, print out the best value for k and the F1-score it achieved.

In [15]:
def find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25):
    best_k = min_k
    best_score = 0 
    for k in range(min_k, max_k+1, 2):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        score = f1_score(y_test, preds)
        if score > best_score:
            best_score = score
            best_k = k      
    print('.-._.-.'*3)
    print('Best k:', best_k,'\n'
         'F1-score:', best_score,)
    print('._.-._.' *3)

In [16]:
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=25)
# Expected Output:

# Best Value for k: 3
# F1-Score: 0.6444444444444444

.-._.-..-._.-..-._.-.
Best k: 11 
F1-score: 0.7943262411347518
._.-._.._.-._.._.-._.


We improved our model performance by over 4 percent just by finding an optimal value for k. Good job! There are other parameters in the model that you can also tune. In a later section, we'll cover how we can automate the parameter search process using a technique called **_Grid Search_**. For, try playing around with the different options for parameters, and seeing how it affects model performance. For a full list of model parameters, see the [sklearn documentation !](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

## (Optional) Level Up: Iterating on the Data

As an optional (but recommended!) exercise, think about the decisions we made during the preprocessing steps that could have affected our overall model performance. For instance, we replaced missing age values with the column median. Could this have affected ourn overall performance? How might the model have fared if we had just dropped those rows, instead of using the column median? What if we reduced dimensionality by ignoring some less important columns altogether?

In the cells below, revisit your preprocessing stage and see if you can improve the overall results of the classifier by doing things differently. Perhaps you should consider dropping certain columns, or dealing with null values differently, or even using a different sort of scaling (or none at all!). Try a few different iterations on the preprocessing and see how it affects the overall performance of the model. The `find_best_k` function handles all of the fitting--use this to iterate quickly as you try different strategies for dealing with data preprocessing! 


In [17]:
#unscaled data
X_train, X_test, y_train, y_test = train_test_split(df_hot, target, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=50)

.-._.-..-._.-..-._.-.
Best k: 19 
F1-score: 0.6052631578947368
._.-._.._.-._.._.-._.


In [18]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df_hot)

scaled_df2 = pd.DataFrame(scaled, columns=df_hot.columns)
scaled_df2.head()

/Users/michaelmoravetz/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,1.0,0.271174,0.125,0.0,0.014151,0.0,0.0,1.0
1,0.0,0.0,0.472229,0.125,0.0,0.139136,1.0,0.0,0.0
2,1.0,0.0,0.321438,0.000,0.0,0.015469,0.0,0.0,1.0
3,0.0,0.0,0.434531,0.125,0.0,0.103644,0.0,0.0,1.0
4,1.0,1.0,0.434531,0.000,0.0,0.015713,0.0,0.0,1.0


In [20]:
#MinMax Scaled is best results so far
X_train, X_test, y_train, y_test = train_test_split(scaled_df2, target, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=100)

.-._.-..-._.-..-._.-.
Best k: 5 
F1-score: 0.7810650887573964
._.-._.._.-._.._.-._.


In [43]:
df2 = pd.read_csv('titanic.csv')
df2.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [44]:
df2.drop(['Cabin', 'Ticket','Name','PassengerId'], axis=1, inplace=True)
df2.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [45]:
df2.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [46]:
criteria = (df2['Sex'] == 'female')
df2.loc[ criteria, 'Sex'] = 0
df2.loc[~criteria, 'Sex'] = 1
df2.loc[df2['Age'] < 16, 'Sex'] = 2
df2.Age.fillna(df2['Age'].mode(),inplace=True)
# df2.loc[df2['Age'] < 16, 'Sex'] = 2
df2.dropna(axis=0,inplace=True) 
df2.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [47]:
hot2 = pd.get_dummies(df2)
labels = hot2.Survived
hot2.drop('Survived', inplace=True, axis=1)
hot2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1


In [48]:
X_train, X_test, y_train, y_test = train_test_split(hot2, labels, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=100)

.-._.-..-._.-..-._.-.
Best k: 71 
F1-score: 0.6222222222222222
._.-._.._.-._.._.-._.


In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(hot2)

hot_scale = pd.DataFrame(scaled, columns=hot2.columns)
hot_scale.head()

/Users/michaelmoravetz/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.5,0.271174,0.2,0.0,0.014151,0.0,0.0,1.0
1,0.0,0.0,0.472229,0.2,0.0,0.139136,1.0,0.0,0.0
2,1.0,0.0,0.321438,0.0,0.0,0.015469,0.0,0.0,1.0
3,0.0,0.0,0.434531,0.2,0.0,0.103644,0.0,0.0,1.0
4,1.0,0.5,0.434531,0.0,0.0,0.015713,0.0,0.0,1.0


In [50]:
X_train, X_test, y_train, y_test = train_test_split(hot_scale, labels, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=100)

.-._.-..-._.-..-._.-.
Best k: 27 
F1-score: 0.7936507936507936
._.-._.._.-._.._.-._.


In [51]:
hot_scale_bin = hot_scale
hot_scale_bin['Age'] = pd.cut(hot_scale_bin['Age'],bins=3)
hot_scale_bin.Fare = pd.cut(hot_scale_bin.Fare, bins=6) 

In [52]:
hot_scale_bin = pd.get_dummies(hot_scale_bin)
hot_scale.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.5,"(-0.001, 0.333]",0.2,0.0,"(-0.001, 0.167]",0.0,0.0,1.0
1,0.0,0.0,"(0.333, 0.667]",0.2,0.0,"(-0.001, 0.167]",1.0,0.0,0.0
2,1.0,0.0,"(-0.001, 0.333]",0.0,0.0,"(-0.001, 0.167]",0.0,0.0,1.0
3,0.0,0.0,"(0.333, 0.667]",0.2,0.0,"(-0.001, 0.167]",0.0,0.0,1.0
4,1.0,0.5,"(0.333, 0.667]",0.0,0.0,"(-0.001, 0.167]",0.0,0.0,1.0


In [40]:
X_train, X_test, y_train, y_test = train_test_split(hot_scale_bin, labels, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=100)

.-._.-..-._.-..-._.-.
Best k: 7 
F1-score: 0.8085106382978724
._.-._.._.-._.._.-._.


In [32]:
from sklearn.preprocessing import RobustScaler

In [33]:
scaler = RobustScaler()
scaled = scaler.fit_transform(hot2)

robust = pd.DataFrame(scaled, columns=hot2.columns)
robust.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0.5,0.0,-0.333333,1.0,0.0,-0.336507,0.0,0.0,0.0
1,-0.5,-1.0,0.555556,1.0,0.0,2.229958,1.0,0.0,-1.0
2,0.5,-1.0,-0.111111,0.0,0.0,-0.309453,0.0,0.0,0.0
3,-0.5,-1.0,0.388889,1.0,0.0,1.501168,0.0,0.0,0.0
4,0.5,0.0,0.388889,0.0,0.0,-0.304443,0.0,0.0,0.0


In [42]:
X_train, X_test, y_train, y_test = train_test_split(robust, labels, test_size= .25)
find_best_k(X_train, y_train, X_test, y_test, min_k=1, max_k=100)

.-._.-..-._.-..-._.-.
Best k: 5 
F1-score: 0.7536231884057971
._.-._.._.-._.._.-._.


## Summary

Good job! This concludes today's section!